## IMPORTACIÓN DE LIBRERÍAS

In [34]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

## LECTURA DE DATASET

In [35]:
df = pd.read_parquet('reviews_google_yelp.parquet')
print('tamaño:', df.shape)
# Reseteamos el índice
df = df.reset_index(drop=True)
df.head(3)

tamaño: (1559364, 7)


,ID_Business,User_ID,Date,Stars,Text,State,APP
0,0x80858087f7190353:0xad8c8f43264160f2,1.1244409414967329e+20,2014-06-24 22:43:04,5,The only time I've been here was when a friend...,California,Google
1,0x80858087f7190353:0xad8c8f43264160f2,1.1604993630531707e+20,2011-02-23 00:00:00,4,Don't quite know what to make of Ame. It's We...,California,Google
2,0x80858087f7190353:0xad8c8f43264160f2,1.0920514467965426e+20,2012-07-02 05:00:03,5,We tried to reserve a table on a Friday night ...,California,Google


In [36]:
df_business = pd.read_parquet('business_google_yelp.parquet')
print('tamaño:', df_business.shape)
df_business.head(3)

tamaño: (58337, 11)


,ID_Business,Name,Address,City,State,Postal_code,Category,Stars,Review_count,Delivery,APP
0,0x8644b5f995211beb:0x489b703481320404,Fresh Off The Truck,Mueller Trailer Eatery,Austin,Texas,78723,"Caterer, Restaurant",5.0,8,1,Google
1,0x8640c7ac849faf11:0x8bf1fc28adff27ae,Tacos Bendicion de Dios,5301 Antoine Dr,Houston,Texas,77091,Restaurant,5.0,1,1,Google
2,0x80c2bf3b638a7d29:0x9d350fcca7b5560e,Pizza Al Forno,6541 Hollywood Blvd,Los Angeles,California,90028,Pizza restaurant,3.3,6,1,Google


## ELIMINACIÓN DE RESEÑAS DEL MISMO USUARIO

In [37]:
# Ordenamos el DataFrame por 'Date' en orden descendente
df_general = df.sort_values(by='Date', ascending=False)

# Eliminamos calificaciones repetidas del mismo lugar por el mismo usuario, conservando la calificación más reciente
df_general = df_general.drop_duplicates(subset=['ID_Business', 'User_ID'])

# Reseteamos el índice
df_general = df_general.reset_index(drop=True)

print('tamaño:', df_general.shape)
df_general.head(3)

tamaño: (1512419, 7)


,ID_Business,User_ID,Date,Stars,Text,State,APP
0,i_kM6kP4u4Wxr3o3_9XN_A,PLyOKlWMEw1SJ0RhEsVtsw,2022-01-19 19:45:43,5,One of the best sushi places I've eaten at!! I...,Indiana,Yelp
1,7ZV31rGcDaT1pUKyvJF9Mg,jCFYwSOZOUs1-3Nxab3GdQ,2022-01-19 19:25:48,5,I am so thankful that the founder of La Boulan...,Indiana,Yelp
2,2SYEGA2fZLdzZQ9lmIfM5Q,NyvEE19AZsgCdnX4Q0fB-w,2022-01-19 19:03:21,5,Came here on a Friday night after a small road...,California,Yelp


## FUNCIÓN PARA OBTENER LOS LUGARES QUE VISITÓ EL USUARIO

In [38]:
def get_business_user(user_id):
    # Se ingresa un 'User_ID' y retorna un DataFrame con los negocios que visitó que tengan mayor o igual a 4 estrellas

    s = df_general[df_general['User_ID']==user_id]
    e = s[s['Stars']>=4]

    return set(e['ID_Business'])

## PRUEBA DE LLAMADA A LA FUNCIÓN

In [39]:
business_user = get_business_user('6s-g2vFu12OemhiK3FJuOQ')
business_user

{'2g3rvWWSD26WQEmK-iYj3A',
 '2s5h1TMhYPdRUUNcQFr3WA',
 '4TleqRlbwgXcIgz33HjLJQ',
 '4hv1n1VBzMvE-T4UtIcOcw',
 '7haOE61-G8VkOfFgS9mWLA',
 '8crVeVL3pi8OFRj9c_3vxQ',
 'BHrKTazoosZdYUm2brFVaA',
 'CftOliRKBvSPABNzt49pdQ',
 'E01_x43vPxeOQ9MFbhSKDQ',
 'EKSmnS-fup3HNFLR9J17mQ',
 'JUlsvVAvZvGHWFfkKm0nlg',
 'O1XwKgUYNI_YBXnOqgaTaw',
 'PeQjA9ciksbpvp4HoU84tQ',
 'PiQVIJI92Z9O37jg9YMyPw',
 'PwHa_x_kUykiJBg1Vo1BiQ',
 'QWqKTWQ2OiDgo3dzNkpung',
 'QXGnmsKYANxOyZBqM-zlJA',
 'Qw7tz-UkPrpXaVidWuab4Q',
 'S8ZFYEgMejpChID8tzKo9A',
 'VJpuuNZ8QXR2zTHIhB0yhw',
 'WK7ZL3iyTjxjBILrnXTjmQ',
 'WltG6PWu0zRTTUpWne3W8w',
 'XvV9B9Y4YfVVehSVkX6Ulg',
 'Y6K9DT44-4Md-ycMkC0nLg',
 'ZRCY1Y3V4ikJs8K7yZ554A',
 'ZRb203peHsQKibqyKqZaGg',
 '_PvljXZHQlUmL8SuOXF8ug',
 'fzNz9cPexB7PuhUGwygbPQ',
 'k4U4QTDGYp_4UEfAR56zCA',
 'k8SvXoV6TFMTfrRsvzJ6uA',
 'mGpX6RCSA-7_EkE2zxl-KQ',
 'nXx6nOAqC0DqVCUObuUsMA',
 'onp9X_9RRAEzA-yqdiSOcA',
 'or_WcjWELDsssk2JruvJcg',
 'pJtvP5aHophr2gjIVvX-YA',
 'pe5ZkiVbgg4GmzPjZ999Lg',
 'pnWpIvIsNnp6S2BW0pyreg',
 

## FUNCIÓN PARA OBTENER EL TOP DE LUGARES DE UN ESTADO

In [40]:
def get_top_business_state1(state):
    top = df_general[df_general['State'] == state]
    top = top.groupby('ID_Business', as_index = False)['Stars'].mean()
    top = set(top[top['Stars']==5]['ID_Business'])
    return top 

In [41]:
def get_top_business_state2(state):
    # Filtramos el dataset por estado:
    top = df_business[df_business['State'] == state]
    # Para evitar un posible sesgo, solo tomaremos negocios que tengan un mínimo de 15 reseñas:
    top = top[top['Review_count']>=15]
    # Por último, nos quedaremos con aquellos negocios que tengan un mínimo de 4.5 estrellas de promedio:
    top = set(top[top['Stars']>=4.5]['ID_Business'])
    
    return top 

In [42]:
# FUNCIÓN PARA OBTENER UNA TABLA DE 5 FILAS DE ID DE NEGOCIOS CON SUS REVIEWS:
def get_tabla_idnegocio_reviews(id_negocios):
    tabla_idnegocio_reviews = df_general[df_general['ID_Business'].isin(id_negocios)]
    
    tabla_idnegocio_reviews = tabla_idnegocio_reviews[tabla_idnegocio_reviews['Stars']>=4.5]
    tabla_idnegocio_reviews = tabla_idnegocio_reviews[tabla_idnegocio_reviews['Text'] != 'No text']

    # Ordenamos el DataFrame por 'ID_Business' y 'Date' de manera descendente:
    tabla_idnegocio_reviews = tabla_idnegocio_reviews.sort_values(by=['ID_Business', 'Date'], ascending=[True, False])

    # Agrupamos por 'ID_Business' y seleccionamos las primeras 5 filas de cada grupo:
    tabla_idnegocio_reviews = tabla_idnegocio_reviews.groupby('ID_Business').head(5)


    tabla_idnegocio_reviews = tabla_idnegocio_reviews[['ID_Business', 'Text']]
    
    return tabla_idnegocio_reviews


In [43]:
get_tabla_idnegocio_reviews(['0IheZIsRS5tejvxocP18mQ'])

,ID_Business,Text
490320,0IheZIsRS5tejvxocP18mQ,There are 3 halal food trucks across from the ...
1014067,0IheZIsRS5tejvxocP18mQ,This food cart is the best one yet. The three ...
1140929,0IheZIsRS5tejvxocP18mQ,They are reliable when you want something quic...
1180604,0IheZIsRS5tejvxocP18mQ,Quick and delicious! There's so many food truc...
1280037,0IheZIsRS5tejvxocP18mQ,I frequently crave this since I've moved out o...


## LLAMANDO A LA FUNCIÓN

In [44]:
top_business_state = get_top_business_state1('Indiana')
top_business_state

{'-788YqeAedOZseDJOo7kfQ',
 '0IheZIsRS5tejvxocP18mQ',
 '0jDPdf7kIkjZuNDNVH8Qew',
 '0x8811871076727871:0xc297aaeecae97a92',
 '0x8811eba41b1ae139:0xbfe6db405bd07870',
 '0x8814db08416a2423:0x575748da63553448',
 '0x886b43f6faa8e699:0x7a9d728e33e33f5e',
 '0x886b50920817545d:0x7d843fcca49569a8',
 '0x886b538da784b4e5:0xa845ca9abd40f8d3',
 '4xcsIgbYhG54418KkAVWow',
 '5CNmglbq_YAAbu7Fsy4PKw',
 '5jXiZ23pb-xf457uNM2yZg',
 '6qHInhFcFigPhTh822qJpg',
 'AxtOW_b6a6iuojs_ObGpzA',
 'Cez_QloMCaeaXfGbKtQ3Pg',
 'ChbVjsP8S_hPb93Qb2t09A',
 'DPih_TVOcKR-CyA6K13j9w',
 'FzsdmAlIGU8xsK4IcdBEMg',
 'MDePpQWpv0vgaO3oi_pxOg',
 'MUQGkZQqPx5dpM6yhajkDQ',
 'OQSihGXRZqKMeW5vSNVwhQ',
 'Psf1p-qzz1nnNlN6RepawQ',
 'Pz8L8B8Ig-0aD21-z_vDQQ',
 'Q0fM4AFRmc35ky8rY28xmA',
 'RK6-cJ9hj53RzOlCBmpT-g',
 'TxMN_VTVsyv_482VHJkfIA',
 'VOi8E3DYV1cHvJy5RinGwA',
 'VhupSRgSHupX87HcRNAcHg',
 'YbpNzwI5iBvsBDwdwc9Mmg',
 '_KqbkPAOI0s2ss0m-ucF5A',
 'hNYgEnEfGAvHyuHuMHsWMQ',
 'ib1OFx4VYp-UUVpwMKVw5A',
 'kUJBWLIOLrCSVd6878h80g',
 'lqQxSrrlTCBn15Y8_

In [45]:
s = get_top_business_state2('Indiana')
e = get_tabla_idnegocio_reviews(s)
print('tamaño:', e.shape)
e.head(3)

tamaño: (6433, 2)


,ID_Business,Text
893,-1XSzguS6XLN-V6MVZMg2A,"After Hurricane Ida, life became a little hect..."
1111,-1XSzguS6XLN-V6MVZMg2A,We had a wonderful experience at Restaurant Re...
1187,-1XSzguS6XLN-V6MVZMg2A,I ate here two nights in a row. The first nigh...


## FUNCIÓN PARA OBTENER LOS STATES DEL User_ID

In [46]:
def get_states(User_ID):
    return list(df_general[df_general['User_ID'] == User_ID]['State'].unique())

## LLAMANDO A LA FUNCIÓN

In [47]:
get_states('6s-g2vFu12OemhiK3FJuOQ')

['California', 'Indiana']

## FUNCIÓN PARA CONVERTIR UNA LISTA DE ID_Business A UNA LISTA DE SUS NOMBRES DE NEGOCIO RESPECTIVOS

In [48]:
# Modificación por Hernán:
def nombre_negocio(lista_ids):
    df_business = pd.read_parquet('business_google_yelp.parquet')
    recomendacion_texto2 = []
    for id in lista_ids:
        s = df_business[df_business['ID_Business'] == id]
        e = s['Name'].to_list()[0]
        recomendacion_texto2.append(e)
    return recomendacion_texto2

## LLAMANDO A LA FUNCIÓN

In [49]:
nombre_negocio(['yX-eHIG--H3geTNWZ2Q6SA',
  'Cez_QloMCaeaXfGbKtQ3Pg',
  '6qHInhFcFigPhTh822qJpg',
  'AxtOW_b6a6iuojs_ObGpzA',
  '-788YqeAedOZseDJOo7kfQ'])

['Crush Gourmet Raspados',
 'Taqueria Los Poblanos',
 'Graze Away Florida Charcuterie Boards with a Twist',
 'Taba Brew',
 'Onur Food Market']

## FUNCIÓN PARA OBTENER LA RESEÑA QUE MEJOR HAYA CALIFICADO UN USUARIO:

In [50]:
# Se ingresa un id de usuario y retorna la reseña que mejor haya calificado 
# (si hay varias entonces se retornará la más reciente):

def get_mejor_calificacion(id_usuario):
    mejor_calificacion = df_general[df_general['User_ID'] == id_usuario]
    mejor_calificacion = mejor_calificacion.sort_values(by = ['Stars', 'Date'], ascending = False)
    mejor_calificacion = mejor_calificacion['Text'].head(1).to_list()[0]
    return mejor_calificacion    

## LLAMANDO A LA FUNCIÓN

In [51]:
df_general[df_general['User_ID']=='jCFYwSOZOUs1-3Nxab3GdQ']

,ID_Business,User_ID,Date,Stars,Text,State,APP
1,7ZV31rGcDaT1pUKyvJF9Mg,jCFYwSOZOUs1-3Nxab3GdQ,2022-01-19 19:25:48,5,I am so thankful that the founder of La Boulan...,Indiana,Yelp
20,hfbZ97Te3T4jeWN6GgsGrQ,jCFYwSOZOUs1-3Nxab3GdQ,2022-01-19 16:36:25,5,I've been wanting to visit Willa Jean for the ...,California,Yelp
200635,QIYVaE6qrgAYHV6aJhqfJQ,jCFYwSOZOUs1-3Nxab3GdQ,2020-12-07 22:13:30,5,"Oh my WORD, the food was phenomenal! We ordere...",California,Yelp
313905,26jYcdFR5oLqKDKpTH0JIQ,jCFYwSOZOUs1-3Nxab3GdQ,2020-06-04 18:47:02,5,Where do I even begin? This is THE place for c...,Indiana,Yelp
539260,IMWoj619K8ezUmFP8ktXEg,jCFYwSOZOUs1-3Nxab3GdQ,2019-08-11 17:24:13,5,If you happen to be near Banh Mi Boys and hung...,California,Yelp
567807,mhrW9O0O5hXGXGnEYBVoag,jCFYwSOZOUs1-3Nxab3GdQ,2019-07-16 16:57:19,3,I wasn't impressed by Jacques-Imo's. The staff...,California,Yelp
573386,FAvEJQ0E7n4f_Wasg9UvEw,jCFYwSOZOUs1-3Nxab3GdQ,2019-07-11 15:16:50,5,This is one of the best Mediterranean places i...,California,Yelp


## FUNCIÓN SIMILITUD DE COSENO

In [52]:
def similitud_coseno(resumen_pelicula, tabla):
    i = tabla

    tfidf = TfidfVectorizer(stop_words="english")
    i["Text"] = i["Text"].fillna("")

    tfidf.fit(i["Text"])
    tfidf_matriz = tfidf.transform([resumen_pelicula])

    coseno_sim = linear_kernel(tfidf_matriz, tfidf.transform(i["Text"]))

    simil = list(enumerate(coseno_sim[0]))
    simil = sorted(simil, key=lambda x: x[1], reverse=True)
    simil = simil[1:11]
    movie_index = [i[0] for i in simil]

    lista = i["ID_Business"].iloc[movie_index].tolist()[:5]

    return lista

## LLAMANDO A LA FUNCIÓN

In [53]:
s = get_mejor_calificacion('jCFYwSOZOUs1-3Nxab3GdQ')
e = get_tabla_idnegocio_reviews(top_business_state)
recomendacion(s, e)

NameError: name 'recomendacion' is not defined

## FUNCIÓN RECOMENDACIÓN

In [ ]:
# La que está actualmente en GitHub:
def recomendacion1(user_id):
    states = get_states(user_id)
    recomendacion_stateS = []
    for state in states:
        top_business = get_top_business_state1(state)
        business_user = get_business_user(user_id)
        recomendacion_state = list(top_business.difference(business_user))[0:5]
        recomendacion_stateS.append(recomendacion_state)
    
    recomendacion_texto = []

    for cantidad in range(len(states)):
        recomendado = nombre_negocio(recomendacion_stateS[cantidad])
        recomendacion_texto_base = "Para el estado de " + states[cantidad] + " te recomendamos los siguientes lugares: " + ", ".join(recomendado)
        recomendacion_texto.append(recomendacion_texto_base)
    
    return {"Recomendación": ". ".join(recomendacion_texto)}

In [ ]:
# La función mejorada en el que se evita el sesgo de la cantidad baja de reseñas:
def recomendacion2(user_id):
    states = get_states(user_id)
    recomendacion_stateS = []
    for state in states:
        top_business = get_top_business_state2(state)
        business_user = get_business_user(user_id)
        recomendacion_state = list(top_business.difference(business_user))[0:5]
        recomendacion_stateS.append(recomendacion_state)
    
    recomendacion_texto = []

    for cantidad in range(len(states)):
        recomendado = nombre_negocio(recomendacion_stateS[cantidad])
        recomendacion_texto_base = "Para el estado de " + states[cantidad] + " te recomendamos los siguientes lugares: " + ", ".join(recomendado)
        recomendacion_texto.append(recomendacion_texto_base)
    
    return {"Recomendación": ". ".join(recomendacion_texto)}

In [ ]:
def recomendacion3(user_id):
    states = get_states(user_id)
    recomendacion_stateS = []
    for state in states:
        top_business = get_top_business_state2(state)
        business_user = get_business_user(user_id)
        recomendacion_state = top_business.difference(business_user) # conjunto que contiene los id de los negocios top menos los que el usuario ya visito
        # se llama a la funcion que con estos id de negocio crea un tabla de 2 columnas donde estan los id de solos estos negocios y sus respectivas
        # reseñas filtradas por estado, cantidad de reseñas minimas y cantidad de estrellas y almacena esta tabla en la variable tabla top negocios

        tabla_idnegocio_reviews = get_tabla_idnegocio_reviews(recomendacion_state)

        # se llama a la funcion que obtiene la mejor reseña que dio el usuario  y se almacena dentro de la variable resumen_pelicula(reseña_top_user)

        
        mejor_calificacion = get_mejor_calificacion(user_id)

        

        # se llama a la funcion similitud de reseñas que da como resultado los id de las reseñas que mejor se parece a la reseña top del usuario,
        # estos resultados se almacenan en una variable y se convierten a conjuntos para eliminar negocios repetido, luego se convierte en una lista
        # tomando los 5 primeros resutados

        recomendacion_state = similitud_coseno(mejor_calificacion, tabla_idnegocio_reviews)
        



        # una vez que se hace la diferencia, los negocios que te queden
        # introducirlos a la funcion para que los ordene de mayor a menor la similitu y de esos escoger los primeros 5

        recomendacion_stateS.append(recomendacion_state)

    recomendacion_texto = []

    for cantidad in range(len(states)):
        recomendado = nombre_negocio(recomendacion_stateS[cantidad])
        recomendacion_texto_base = "Para el estado de " + states[cantidad] + " te recomendamos los siguientes lugares: " + ", ".join(recomendado)
        recomendacion_texto.append(recomendacion_texto_base)

    return {"Recomendación": ". ".join(recomendacion_texto)}

## LLAMANDO A LA FUNCIÓN

In [ ]:
x = recomendacion1('jCFYwSOZOUs1-3Nxab3GdQ')
x

{'Recomendación': "Para el estado de Indiana te recomendamos los siguientes lugares: Sweet Elizabeth's Organics, Gordon Stein -I Love Philly Homes Team - Compass, LJ Thai Sushi, Molecular Munchies, Litehouse Whole Food Grill Hobart. Para el estado de California te recomendamos los siguientes lugares: Veg & Go Plant Based Fast Food, Elegance On Display, Pizza Pasta Pomodoro, Pure Juice & Smoothie Lounge, Fuji Japanese Restaurant"}

In [ ]:
x = recomendacion2('jCFYwSOZOUs1-3Nxab3GdQ')
x

{'Recomendación': 'Para el estado de Indiana te recomendamos los siguientes lugares: Sherwood Cafe Family Restaurant, Whisky Lounge, Botanas locas, Valley, Three One Seven Salon LLC. Para el estado de California te recomendamos los siguientes lugares: WHITE HORSE CAFE, Korean Express Restaurant, Pocha LA, Ronan, Kazoku Sushi'}

In [ ]:
x = recomendacion3('jCFYwSOZOUs1-3Nxab3GdQ')
x

{'Recomendación': "Para el estado de Indiana te recomendamos los siguientes lugares: Tacos Pipeye, Yamato, Perenn Bakery, Smoothie King, Perenn Bakery. Para el estado de California te recomendamos los siguientes lugares: Hanabishi Japanese Cuisine, SF Honey Pig Korean BBQ Restaurant, Companion Early Bird Outlet, Hiko Sushi, Sophia's Donuts & Sandwiches"}

### Observación: Si bien se hizo uso del dataset reviews_google_yelp.parquet (alrededor de 1 millón y medio de filas), para el sistema de recomendación solo se usará una muestra aleatoria, asdf2.parquet